In [2]:
import pandas as pd;
import numpy as np;
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
### preprocessing data before going for vectorization

## colum_names = data_frame.columns
# data_frame = data_frame.drop('Unnamed: 0', axis=1)
# data_frame = data_frame.drop('label', axis = 1)
# data_frame['text'] = data_frame['text'].str.replace('Subject:','')
# data_frame['text'] = data_frame['text'].str.lower();
# data_frame.to_csv("emails2.csv");


In [9]:
### use scikit-learn to do vectorization of the text
data_frame = pd.read_csv("emails2.csv")


vectorizer = TfidfVectorizer()

X_features = vectorizer.fit_transform(data_frame['text'])

#converts sparse matrix to dense matrix
X_features = X_features.toarray()

feature_names = vectorizer.get_feature_names_out()


print("Feature Names:", feature_names)
print("TF-IDF Matrix:", X_features)

Feature Names: ['00' '000' '0000' ... 'zzocb' 'zzso' 'zzsyt']
TF-IDF Matrix: [[0.0995327 0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 ...
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]


AttributeError: 'TfidfVectorizer' object has no attribute 'get_feature_names'

In [ ]:
data_frame = data_frame.to_numpy();
print(data_frame.shape);

In [ ]:
X_features = data_frame[: , 1:30];
Y = data_frame[:, 3001];

In [ ]:
# print(X_features);

In [ ]:
## steps in logistic regression
# 1. get the data set
# 2. calculate z  = w*x + b where z is an numpy array
# 3. sigmoid function g(z) = 1 / 1 + e^-z
# 4. write the cost function J(w) = 1/m * summation(-y*(log(a)) - (1 - y)*log(1-a)) where a is sigmoid function and y is the actual output
# 5. write gradient descent algorithm to reduce cost function

In [ ]:
def sigmoid(z):
    '''
      it will return the sigmoid function    
      z = w.x + b
    '''
    z = np.clip(z, -500, 500);
    denominator = 1 + np.exp(-1*z)
    g = 1 / denominator
    
    return g



In [ ]:
# computing cost function 
def compute_cost(W, X_train, Y, b, *args):
    '''computes the sum of the lose'''
    
    (m,n) = X_train.shape;
    totalcost = 0;
    
    for i in range(m):
        z_i = np.dot(W, X_train[i]) + b
        f_wb_i = sigmoid(z_i)
        loss_i = -1*Y[i]*np.log(f_wb_i + 1e-10) - (1 - Y[i])*np.log(1 - f_wb_i + 1e-10)
        totalcost += loss_i
    totalcost = (1/ m) * totalcost
    return totalcost
    

In [ ]:
(m,n) = X_features.shape;
# initial_w = np.random.rand(n);
initial_w = np.zeros(n);
#print(initial_w)
initial_b = 100
print(f"cost is : {compute_cost(initial_w, X_features, Y, initial_b):.3f}");

In [ ]:
def compute_gradient(X, y, W, b, *args):
    '''computes the gradient descent with respect to each w and b'''
    (m, n) = X.shape;
    dj_dw = np.zeros(n);
    dj_db = 0
    
    for i in range(m):
        z_i = np.dot(W, X[i]) + b
        f_xi = sigmoid(z_i)
        for j in range(n):
            dj_dw[j]+= (f_xi - y[i])*X[i][j]
        dj_db += (f_xi - y[i])
    dj_dw = dj_dw / m;
    dj_db = dj_db / m;
    
    return dj_dw, dj_db;

In [ ]:
dj_dw, dj_db = compute_gradient(X_features, Y, initial_w, initial_b)
# print(f"dj_db for initial value w and b is {dj_db}")
#print(f"dj_dw for initial value w and b is {dj_dw.tolist()}");

In [ ]:
def gradient_descent(X, y, W, b, computecost, gradient, alpha, numiters):
    ''' computes best value for W and b and also compute the cost and J_history'''
    
    (m, n) = X.shape;
    
    J_history = [] # it holds the details of the previous transactions
    W_history = []
    
    for i in range(numiters):
        dj_dw, dj_db = gradient(X, y, W, b)
        W = W - alpha * dj_dw;
        b = b - alpha * dj_db;
        
        if i<10000:
            cost = computecost(W, X, y, b);
            J_history.append(cost)
        if i % 10 == 0:
            W_history.append(W);
            print(f"cost for current w is {J_history[-1]}");
    
    return W, b, J_history, W_history;
            

In [ ]:
iterations = 1000
alpha = 0.001

w,b, J_history,_ = gradient_descent(X_features ,Y, initial_w, initial_b, 
                                   compute_cost, compute_gradient, alpha, iterations);